<font color="white">.</font> | <font color="white">.</font> | <font color="white">.</font>
-- | -- | --
![NASA](http://www.nasa.gov/sites/all/themes/custom/nasatwo/images/nasa-logo.svg) | <h1><font size="+3">ASTG Python Courses</font></h1> | ![NASA](https://www.nccs.nasa.gov/sites/default/files/NCCS_Logo_0.png)

---

<center>
    <h1><font color="red">Image Classification with Tensorflow (GPU)</font></h1>
</center>

## Useful Reference

- <a href="https://www.mygreatlearning.com/blog/what-is-tensorflow-machine-learning-library-explained/">What is TensorFlow? The Machine Learning Library Explained</a>
- <a href="https://www.tensorflow.org/tutorials/keras/regression">Basic regression: Predict fuel efficiency</a>
- <a href="https://stackabuse.com/tensorflow-2-0-solving-classification-and-regression-problems/">Tensorflow 2.0: Solving Classification and Regression Problems</a>
- <a href="https://www.toptal.com/machine-learning/tensorflow-machine-learning-tutorial">Getting Started with TensorFlow: A Machine Learning Tutorial</a>
- <a href="https://sebastianraschka.com/faq/docs/tensorflow-vs-scikitlearn.html">What is the main difference between TensorFlow and scikit-learn?</a>
- <a href="https://adventuresinmachinelearning.com/python-tensorflow-tutorial/">Python TensorFlow Tutorial – Build a Neural Network</a>
- <a href="https://steadforce.com/en/first-steps-tensorflow-part-3/">A simple neural network with TensorFlow</a>

# <font color="red"> GPUs</font>

![GPUs](http://www.nvidia.com/docs/IO/143716/how-gpu-acceleration-works.png)
Image Source: NVIDIA

- Graphics Processing Units (GPUs) are custom designed to be very efficient at handling computer graphics and image processing.
- Central Processing Units (CPUs) handle computations serially, meaning the logic in handled in one stream: the next task will complete when the subsequent task has finished. CPUs can execute tasks in parallel across cores. For example, most computer CPUs tend to have either two, four or six cores.
- In comparison, GPUs have hundreds of 'cores'. This massively parallel architecture is what gives the GPU its high compute performance.

**Useful Terminology**

| Term | Meaning |
| ---  | --- |
| `host` | the CPU |
| `device` | the GPU |
| `host memory` | the system main memory |
| `device memory` | onboard memory on a GPU card |
| `kernels` | a GPU function launched by the host and executed on the device |
| `device function` | a GPU function executed on the device which can only be called from the device  |

## Accessing the GPU on Google Colab

In order to access GPUs for free:

1. Go to the `Runtime` menu,
2. Click on `Change runtime type`, and 
3. In the pop-up box, under `Hardware accelerator`, select `GPU` and click on `SAVE`.


### Load the modules

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%matplotlib inline
import sys
import csv
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

print(tf.__version__)

# <font color="red">Image Classification</font> 

We use the [MNIST data set](http://yann.lecun.com/exdb/mnist/) (Modified National Institute of Standards and Technology database).

* Is a large database of handwritten digits that is commonly used for training various image processing systems.
* The database is also widely used for training and testing in the field of machine learning.
* The dataset we will be using contains 70000 images of handwritten digits among which 10000 are reserved for testing.
* It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting.

### Check GPU Availability in Tensorflow

In [ ]:
#https://www.kaggle.com/hassanamin/tensorflow-mnist-gpu-tutorial

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

### Listing Devices including GPU's with Tensorflow

In [ ]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

### Check GPU in Tensorflow

In [ ]:
tf.test.is_gpu_available()

## <font color="blue"> Load MNiST Dataset</font>

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
print("Shape train inputs:  ", x_train.shape)
print("Shape train outputs: ", y_train.shape)
print("Shape test  inputs:  ", x_test.shape)
print("Shape test  outputs: ", y_test.shape)

In [ ]:
print("Type train inputs:  ", x_train.dtype)
print("Type train outputs: ", y_train.dtype)
print("Type test  inputs:  ", x_test.dtype)
print("Type test  outputs: ", y_test.dtype)

In [ ]:
np.unique(x_train)

In [ ]:
np.unique(x_test)

In [ ]:
np.unique(y_train)

In [ ]:
np.unique(y_test)

## <font color="blue"> Preprocess the Training and Test Datasets</font>

Change the type from integer to floating point. This will reduce our memory requirements by forcing the precision of the pixel values to be 32 bit, the default precision used by Keras anyway.

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

Normalize the data:

In [ ]:
x_train = x_train / 255.0
x_test = x_test / 255.0

- The training and test datasets are structured as a 3-dimensional array of instance, image width and image height. 
- For a multi-layer perceptron model we must reduce the images down into a vector of pixels. In this case the 28×28 sized images will be 784 pixel input values.
- We can do this transform easily using the `reshape()` function on the NumPy array.

In [ ]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

**Convert class vectors to binary class matrices**

In [ ]:
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

## <font color="blue"> Create Sequential Model Using Tensorflow Keras</font>

Architecture of the Network is:

1. Input layer for 28x28=784 images in MNiST dataset
2. Dense layer with 128 neurons and ReLU activation function
2. Dense layer with 128 neurons and ReLU activation function
3. Output layer with 10 neurons for classification of input images as one of ten digits(0 to 9)

In [ ]:
mnist_model = tf.keras.models.Sequential()
mnist_model.add(tf.keras.layers.Dense(128, activation='relu', 
                                input_shape=(784,)))
#mnist_model.add(tf.keras.layers.Dropout(0.2))
#mnist_model.add(tf.keras.layers.Dense(128, activation='relu'))
mnist_model.add(tf.keras.layers.Dropout(0.2))
mnist_model.add(tf.keras.layers.Dense(10, activation='softmax'))

In [ ]:
mnist_model.summary()

### Compile the Model Designed Earlier

Before the model is ready for training, it needs a few more settings. These are added during the model's compile step:

- Loss function This measures how accurate the model is during training. You want to minimize this function to "steer" the model in the right direction.
- Optimizer This is how the model is updated based on the data it sees and its loss function.
- Metrics Used to monitor the training and testing steps. The following example uses accuracy, the fraction of the images that are correctly classified.

In [ ]:
mnist_model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(),
              metrics=['accuracy'])

## <font color="blue"> Training and Validation</font>

The `mnist_model.fit` method adjusts the model parameters to minimize the loss:

In [ ]:
num_epochs = 5
batch_size = 16

In [ ]:
history = mnist_model.fit(x_train, y_train,
                          batch_size = batch_size,
                          epochs = num_epochs,
                          verbose = 1,
                          validation_data = (x_test, y_test))

## <font color="blue"> Plot the Deceasing Loss over Epochs</font>

Use Pandas to plot a graph showing the decrease in mean squared error (mse) as training improves the model.

In [ ]:
loss_df = pd.DataFrame(history.history)
loss_df.plot()

## <font color="blue"> Evaluate the Model</font>

The `mnist_model.evaluate` method checks the models performance, usually on a "Validation-set" or "Test-set".

In [ ]:
score = mnist_model.evaluate(x_test,  y_test, verbose=0)
print('Test loss:     {}'.format(score[0]))
print('Test accuracy: {}'.format(score[1]))

## <font color="blue"> Visualize Predictions</font>

In [ ]:
probabilities = mnist_model.predict(x_test, steps=1)
predicted_labels = np.argmax(probabilities, axis=1)

In [ ]:
def display_digits(X, y):
    """
      Given an array of images of digits X and 
      the corresponding values of the digit y,
      this function plots the first 96 images and their values.
    """
    # Figure size (width, height) in inches
    fig = plt.figure(figsize=(8, 6))

    # Adjust the subplots 
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

    for i in range(96):
        # Initialize the subplots: 
        #    Add a subplot in the grid of 8 by 12, at the i+1-th position
        ax = fig.add_subplot(8, 12, i + 1, xticks=[], yticks=[])
        
        # Display an image at the i-th position
        ax.imshow(X[i].reshape(28, 28), cmap=plt.cm.binary, interpolation='nearest')
       
        # label the image with the target value
        ax.text(0, 7, str(y[i]))

    # Show the plot
    plt.show()

In [ ]:
display_digits(x_test, predicted_labels)

## <font color="blue">Save the Model</font>

In [ ]:
mnist_model.save('my_MNIST_model')

Then to reload the model later, we can use this:

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('my_MNIST_model')

In [ ]:
# https://www.tensorflow.org/tutorials/quickstart/advanced
# https://liufuyang.github.io/2017/04/01/just-another-tensorflow-beginner-guide-3.html